# Transformer encoder

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math

import numpy as np
import pandas as pd

import torch
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.optim import AdamW

import lightning as L
from lightning.pytorch import seed_everything
from lightning.pytorch.callbacks import RichProgressBar
from lightning.pytorch import loggers as pl_loggers
from lightning.pytorch.callbacks import ModelCheckpoint

In [3]:
from data_preprocessor.data_preprocessor import CompositeDataPreprocessor, CompositeGroupedDataPreprocessor, ReduceMemUsageDataPreprocessor, FillNaPreProcessor
from data_preprocessor.feature_engineering import (
    BasicFeaturesPreprocessor,
    DupletsTripletsPreprocessor,
    MovingAvgPreProcessor,
    RemoveIrrelevantFeaturesDataPreprocessor,
    DropTargetNADataPreprocessor,
    FarNearPriceFillNaPreprocessor,
    MovingAvgFillNaPreprocessor,
    RemoveRecordsByStockDateIdPreprocessor,
)
from data_preprocessor.stock_feature_engineering import (
    StockNormalizeFeaturesPreprocessor,
)
from data_preprocessor.polynomial_features import PolynomialFeaturesPreProcessor

from data_generator.data_generator import DefaultTrainEvalDataGenerator, ManualKFoldDataGenerator, TimeSeriesKFoldDataGenerator

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x14826585e520>
Traceback (most recent call last):
  File "/userhome/cs2/tsangsyf/anaconda3_2/lib/python3.11/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/userhome/cs2/tsangsyf/anaconda3_2/lib/python3.11/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/userhome/cs2/tsangsyf/anaconda3_2/lib/python3.11/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
                   ^^^^^^^^^^^^^^^^^^
  File "/userhome/cs2/tsangsyf/anaconda3_2/lib/python3.11/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
             ^^^^^^^^^^^^^^^^

In [4]:
# https://lightning.ai/docs/pytorch/stable/common/trainer.html#reproducibility
seed_everything(42, workers=True)

[rank: 0] Seed set to 42


42

# Hyperparameters / Setup

In [5]:
checkpoint_dir = "transformer_encoder_by_time_checkpoints/20240425_transformer_dupletstriplets"

In [6]:
model_folder_version_name = None
# change to a string for a specific name
# model_folder_version_name = "test"

# https://lightning.ai/docs/pytorch/stable/api/lightning.pytorch.loggers.tensorboard.html
# https://lightning.ai/docs/pytorch/stable/extensions/logging.html
tb_logger = pl_loggers.TensorBoardLogger(".", version=model_folder_version_name)

## Train-validation split (align with LightGBM experiment/setup for fair comparison)

In [7]:
N_fold = 5
time_series_k_fold_data_generator = TimeSeriesKFoldDataGenerator(n_fold=N_fold, test_set_ratio=0.1)

## Data hyperparameters

In [8]:
prediction_length = 1
seq_len = 55

training_batch_size = 256
validation_batch_size = 256

## Model hyperparameters

In [9]:
embedding_dim = 4
d_model = 32
nhead = 4
d_hid = 32
nlayers = 2
dropout = 0.1

## Training hyperparameters

In [10]:
num_epochs = 10
limit_train_batches = 1.0
# limit_train_batches = 2
learning_rate = 1e-3
gradient_clip_val = 0.5

# Model

TODO:
1. use input_ff_sigmoid?
2. any further model enhancement?

In [11]:
class TransformerModel(nn.Module):
    def __init__(
        self,
        num_input_features: int,
        num_classes: int,
        embedding_dim: int,
        d_model: int,
        nhead: int,
        d_hid: int,
        nlayers: int,
        dropout: float,
    ):
        super().__init__()
        self.model_type = 'Transformer'
        self.embedding = nn.Embedding(num_classes, embedding_dim)
        self.input_ff = nn.Linear(num_input_features + embedding_dim, d_model)
        self.input_ff_sigmoid = nn.Sigmoid()
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout, batch_first=True)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.d_model = d_model
        self.final_linear = nn.Linear(d_model, 1)

    def forward(
        self,
        src: Tensor,
        item_ids: Tensor,
        src_mask: Tensor = None,
    ) -> Tensor:
        # src: [batch_size b, seq_len k 200, features 27]
        # item_ids: [batch_size b, seq_len 200]
        batch_size = src.size(dim=0)
        seq_len = src.size(dim=1)
        num_input_features = src.size(dim=2)
        output = src
        embedded = self.embedding(item_ids)
        # embedded: [batch_size b, seq_len 200, embedding_dim]
        # idea from https://github.com/huggingface/transformers/blob/v4.38.2/src/transformers/models/time_series_transformer/modeling_time_series_transformer.py#L1290
        # embedded = embedded.unsqueeze(dim=1)
        # embedded: [batch_size b, 1, embedding_dim]
        # embedded = embedded.expand(-1, seq_len, -1)
        # embedded: [batch_size b, seq_len k 55, embedding_dim]

        output = torch.cat((src, embedded), dim=-1)
        # [batch_size b, seq_len k 55, features 27 + embedding_dim]

        output = self.input_ff(output)
        # [batch_size b, seq_len k 55, d_model]

        # TODO: do we need sigmoid?
        output = self.input_ff_sigmoid(output)

        # if src_mask is None:
        #     """Generate a square causal mask for the sequence. The masked positions are filled with float('-inf').
        #     Unmasked positions are filled with float(0.0).
        #     """
        #     src_mask = nn.Transformer.generate_square_subsequent_mask(seq_len)
        #     # no "to device" for lightning
        #     # https://pytorch.org/docs/stable/generated/torch.Tensor.to.html#torch.Tensor.to
        #     src_mask = src_mask.to(output)
        #     # src_mask = squared (triangle matrix) matrix [seq_len k 55, seq_len k 55]

        output = self.transformer_encoder(output, src_mask)
        # [batch_size b, seq_len k 200, d_model]

        output = self.final_linear(output)
        # [batch_size b, seq_len k 200, 1]

        # # take the "last" prediction, which includes all previous information
        # output = output[:, -1, :]
        # # [batch_size b, 1]
        output = output.squeeze(dim=2)
        # [batch_size b, , seq_len k 200]

        return output

In [12]:
# https://stackoverflow.com/questions/49433936/how-do-i-initialize-weights-in-pytorch
# https://pytorch.org/docs/stable/nn.init.html#torch.nn.init.xavier_uniform_
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

## Load data

In [13]:
df = pd.read_csv(
    "../optiver-trading-at-the-close/train.csv",
    dtype={
        # stock_id should be int64 / long for embedding
        "date_id": np.float32,
        "seconds_in_bucket": np.float32,
        "imbalance_size": np.float32,
        "imbalance_buy_sell_flag": np.float32,
        "reference_price": np.float32,
        "matched_size": np.float32,
        "far_price": np.float32,
        "near_price": np.float32,
        "bid_price": np.float32,
        "bid_size": np.float32,
        "ask_price": np.float32,
        "ask_size": np.float32,
        "wap": np.float32,
        "target": np.float32,
        "time_id": np.int64,
    },
    usecols=[
        "stock_id",
        "date_id",
        "seconds_in_bucket",
        "imbalance_size",
        "imbalance_buy_sell_flag",
        "reference_price",
        "matched_size",
        "far_price",
        "near_price",
        "bid_price",
        "bid_size",
        "ask_price",
        "ask_size",
        "wap",
        "target",
        "time_id",
    ]
)
raw_df = df.copy(deep=True)

In [14]:
# deep copy for easy reference to raw df without reloading from csv
df = raw_df.copy(deep=True)

In [15]:
display(df)
print(df.dtypes)

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id
0,0,0.0,0.0,3.180603e+06,1.0,0.999812,13380277.00,NaN,NaN,0.999812,60651.500000,1.000026,8493.030273,1.000000,-3.029704,0
1,1,0.0,0.0,1.666039e+05,-1.0,0.999896,1642214.25,NaN,NaN,0.999896,3233.040039,1.000660,20605.089844,1.000000,-5.519986,0
2,2,0.0,0.0,3.028799e+05,-1.0,0.999561,1819368.00,NaN,NaN,0.999403,37956.000000,1.000298,18995.000000,1.000000,-8.389950,0
3,3,0.0,0.0,1.191768e+07,-1.0,1.000171,18389746.00,NaN,NaN,0.999999,2324.899902,1.000214,479032.406250,1.000000,-4.010201,0
4,4,0.0,0.0,4.475500e+05,-1.0,0.999532,17860614.00,NaN,NaN,0.999394,16485.539062,1.000016,434.100006,1.000000,-7.349849,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480.0,540.0,2.440723e+06,-1.0,1.000317,28280362.00,0.999734,0.999734,1.000317,32257.039062,1.000434,319862.406250,1.000328,2.310276,26454
5237976,196,480.0,540.0,3.495105e+05,-1.0,1.000643,9187699.00,1.000129,1.000386,1.000643,205108.406250,1.000900,93393.070312,1.000819,-8.220077,26454
5237977,197,480.0,540.0,0.000000e+00,0.0,0.995789,12725436.00,0.995789,0.995789,0.995789,16790.660156,0.995883,180038.312500,0.995797,1.169443,26454
5237978,198,480.0,540.0,1.000899e+06,1.0,0.999210,94773272.00,0.999210,0.999210,0.998970,125631.718750,0.999210,669893.000000,0.999008,-1.540184,26454


stock_id                     int64
date_id                    float32
seconds_in_bucket          float32
imbalance_size             float32
imbalance_buy_sell_flag    float32
reference_price            float32
matched_size               float32
far_price                  float32
near_price                 float32
bid_price                  float32
bid_size                   float32
ask_price                  float32
ask_size                   float32
wap                        float32
target                     float32
time_id                      int64
dtype: object


In [16]:
k_fold_train_dfs, k_fold_val_dfs, k_fold_num_train_val_sets = time_series_k_fold_data_generator.generate(df)
k_fold_train_df = k_fold_train_dfs[-1]
k_fold_val_df = k_fold_val_dfs[-1]
k_fold_train_df_index = k_fold_train_df.index
k_fold_val_df_index = k_fold_val_df.index
print(len(k_fold_train_dfs), len(k_fold_val_dfs), k_fold_num_train_val_sets, N_fold)
assert k_fold_num_train_val_sets == N_fold and len(k_fold_train_dfs) == N_fold and len(k_fold_val_dfs) == N_fold
print(k_fold_train_df.shape, k_fold_val_df.shape, df.shape)
assert k_fold_train_df.shape[0] + k_fold_val_df.shape[0] == df.shape[0]

5 5 5 5
(4714182, 16) (523798, 16) (5237980, 16)


## Data pre-processing and features

In [17]:
processors = [
    RemoveRecordsByStockDateIdPreprocessor([
        {"stock_id": 19, "date_id": 438},
        {"stock_id": 101, "date_id": 328},
        {"stock_id": 131, "date_id": 35},
        {"stock_id": 158, "date_id": 388},
    ]),
    FarNearPriceFillNaPreprocessor(),
    # ReduceMemUsageDataPreprocessor(verbose=True),
    # BasicFeaturesPreprocessor(),
    DupletsTripletsPreprocessor(enable_triplets=False),
    # MovingAvgPreProcessor("wap"),
    # MovingAvgFillNaPreprocessor("wap", 1.0),
    # StockIdFeaturesPreProcessor(),
    # DropTargetNADataPreprocessor(),
    # RemoveIrrelevantFeaturesDataPreprocessor(['stock_id', 'date_id','time_id', 'row_id']),
    # FillNaPreProcessor(),
    # PolynomialFeaturesPreProcessor(),
]
processor = CompositeDataPreprocessor(processors)

In [18]:
df = processor.apply(df)

CompositeDataPreprocessor - original df shape: (5237980, 16)
Processing RemoveRecordsByStockDateIdPreprocessor...
RemoveRecordsByStockDateIdPreprocessor - removing 220 records
RemoveRecordsByStockDateIdPreprocessor took 0.56s. New df shape: (5237760, 16).
Processing FarNearPriceFillNaPreprocessor...
FarNearPriceFillNaPreprocessor took 0.05s. New df shape: (5237760, 16).
Processing DupletsTripletsPreprocessor...
DupletsTripletsPreprocessor took 1.41s. New df shape: (5237760, 31).
CompositeDataPreprocessor - final df shape: (5237760, 31)


In [19]:
print(df.columns)
display(df)

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id', 'reference_price_far_price_imb',
       'reference_price_near_price_imb', 'reference_price_ask_price_imb',
       'reference_price_bid_price_imb', 'reference_price_wap_imb',
       'far_price_near_price_imb', 'far_price_ask_price_imb',
       'far_price_bid_price_imb', 'far_price_wap_imb',
       'near_price_ask_price_imb', 'near_price_bid_price_imb',
       'near_price_wap_imb', 'ask_price_bid_price_imb', 'ask_price_wap_imb',
       'bid_price_wap_imb'],
      dtype='object')


,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,...,far_price_near_price_imb,far_price_ask_price_imb,far_price_bid_price_imb,far_price_wap_imb,near_price_ask_price_imb,near_price_bid_price_imb,near_price_wap_imb,ask_price_bid_price_imb,ask_price_wap_imb,bid_price_wap_imb
0,0,0.0,0.0,3.180603e+06,1.0,0.999812,13380277.00,1.000000,1.000000,0.999812,...,0.000000,-0.000013,9.400536e-05,0.000000,-0.000013,9.400536e-05,0.000000,0.000107,0.000013,-9.400536e-05
1,1,0.0,0.0,1.666039e+05,-1.0,0.999896,1642214.25,1.000000,1.000000,0.999896,...,0.000000,-0.000330,5.200776e-05,0.000000,-0.000330,5.200776e-05,0.000000,0.000382,0.000330,-5.200776e-05
2,2,0.0,0.0,3.028799e+05,-1.0,0.999561,1819368.00,1.000000,1.000000,0.999403,...,0.000000,-0.000149,2.985892e-04,0.000000,-0.000149,2.985892e-04,0.000000,0.000448,0.000149,-2.985892e-04
3,3,0.0,0.0,1.191768e+07,-1.0,1.000171,18389746.00,1.000000,1.000000,0.999999,...,0.000000,-0.000107,5.066397e-07,0.000000,-0.000107,5.066397e-07,0.000000,0.000107,0.000107,-5.066397e-07
4,4,0.0,0.0,4.475500e+05,-1.0,0.999532,17860614.00,1.000000,1.000000,0.999394,...,0.000000,-0.000008,3.030921e-04,0.000000,-0.000008,3.030921e-04,0.000000,0.000311,0.000008,-3.030921e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480.0,540.0,2.440723e+06,-1.0,1.000317,28280362.00,0.999734,0.999734,1.000317,...,0.000000,-0.000350,-2.914891e-04,-0.000297,-0.000350,-2.914891e-04,-0.000297,0.000059,0.000053,-5.481860e-06
5237976,196,480.0,540.0,3.495105e+05,-1.0,1.000643,9187699.00,1.000129,1.000386,1.000643,...,-0.000128,-0.000385,-2.569161e-04,-0.000345,-0.000257,-1.284415e-04,-0.000216,0.000128,0.000040,-8.791219e-05
5237977,197,480.0,540.0,0.000000e+00,0.0,0.995789,12725436.00,0.995789,0.995789,0.995789,...,0.000000,-0.000047,0.000000e+00,-0.000004,-0.000047,0.000000e+00,-0.000004,0.000047,0.000043,-4.010383e-06
5237978,198,480.0,540.0,1.000899e+06,1.0,0.999210,94773272.00,0.999210,0.999210,0.998970,...,0.000000,0.000000,1.201233e-04,0.000101,0.000000,1.201233e-04,0.000101,0.000120,0.000101,-1.903312e-05


In [20]:
not_include_in_feat = ['stock_id', 'target', 'time_id']

In [21]:
feat_dynamic_real = [item for item in df.columns if item not in not_include_in_feat]
feat_dynamic_real

['date_id',
 'seconds_in_bucket',
 'imbalance_size',
 'imbalance_buy_sell_flag',
 'reference_price',
 'matched_size',
 'far_price',
 'near_price',
 'bid_price',
 'bid_size',
 'ask_price',
 'ask_size',
 'wap',
 'reference_price_far_price_imb',
 'reference_price_near_price_imb',
 'reference_price_ask_price_imb',
 'reference_price_bid_price_imb',
 'reference_price_wap_imb',
 'far_price_near_price_imb',
 'far_price_ask_price_imb',
 'far_price_bid_price_imb',
 'far_price_wap_imb',
 'near_price_ask_price_imb',
 'near_price_bid_price_imb',
 'near_price_wap_imb',
 'ask_price_bid_price_imb',
 'ask_price_wap_imb',
 'bid_price_wap_imb']

In [22]:
# feat_dynamic_real = [
#     "date_id",
#     "seconds_in_bucket",
#     "imbalance_size",
#     "reference_price",
#     "matched_size",
#     "far_price",
#     "near_price",
#     "bid_price",
#     "bid_size",
#     "ask_price",
#     "ask_size",
#     "wap",
#     # "wap_mov_avg_3_1",
#     # "wap_mov_avg_6_3",
#     # "wap_mov_avg_12_6",
#     # "wap_mov_avg_24_12",
# ]
num_input_features = len(feat_dynamic_real)
num_classes = 200
print(num_input_features, num_classes)

28 200


In [23]:
# should not have any na features
any_na_values_mask = df[feat_dynamic_real].isna().any(axis=1)
print(any_na_values_mask.shape, any_na_values_mask[any_na_values_mask].shape)
assert any_na_values_mask[any_na_values_mask].shape[0] == 0

(5237760,) (0,)


In [24]:
df[any_na_values_mask]

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,...,far_price_near_price_imb,far_price_ask_price_imb,far_price_bid_price_imb,far_price_wap_imb,near_price_ask_price_imb,near_price_bid_price_imb,near_price_wap_imb,ask_price_bid_price_imb,ask_price_wap_imb,bid_price_wap_imb


In [25]:
df = df.set_index(["stock_id", "time_id"])
stock_id_list = np.arange(200, dtype=np.int64)
time_id_list = np.arange(26455, dtype=np.int64)
pd_multiindex = pd.MultiIndex.from_product([stock_id_list, time_id_list], names=("stock_id", "time_id"))
df = df.reindex(pd_multiindex, fill_value=0.0)
df = df.reset_index()
display(df)

,stock_id,time_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,...,far_price_near_price_imb,far_price_ask_price_imb,far_price_bid_price_imb,far_price_wap_imb,near_price_ask_price_imb,near_price_bid_price_imb,near_price_wap_imb,ask_price_bid_price_imb,ask_price_wap_imb,bid_price_wap_imb
0,0,0,0.0,0.0,3180602.750,1.0,0.999812,13380277.0,1.000000,1.000000,...,0.000000,-0.000013,0.000094,0.000000,-0.000013,0.000094,0.000000,0.000107,0.000013,-0.000094
1,0,1,0.0,10.0,1299772.750,1.0,1.000026,15261107.0,1.000000,1.000000,...,0.000000,-0.000013,0.000094,0.000054,-0.000013,0.000094,0.000054,0.000107,0.000067,-0.000040
2,0,2,0.0,20.0,1299772.750,1.0,0.999919,15261107.0,1.000000,1.000000,...,0.000000,0.000041,0.000094,0.000079,0.000041,0.000094,0.000079,0.000054,0.000039,-0.000015
3,0,3,0.0,30.0,1299772.750,1.0,1.000133,15261107.0,1.000000,1.000000,...,0.000000,-0.000067,-0.000013,-0.000042,-0.000067,-0.000013,-0.000042,0.000054,0.000024,-0.000030
4,0,4,0.0,40.0,1218204.375,1.0,1.000455,15342675.0,1.000000,1.000000,...,0.000000,-0.000227,-0.000121,-0.000158,-0.000227,-0.000121,-0.000158,0.000107,0.000069,-0.000038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5290995,199,26450,480.0,500.0,2661783.500,-1.0,1.002764,23969216.0,1.000859,1.001494,...,-0.000317,-0.001109,-0.000951,-0.000986,-0.000792,-0.000634,-0.000669,0.000159,0.000123,-0.000035
5290996,199,26451,480.0,510.0,2658917.500,-1.0,1.002764,23972082.0,1.000859,1.001494,...,-0.000317,-0.001109,-0.000951,-0.001055,-0.000792,-0.000634,-0.000738,0.000159,0.000054,-0.000105
5290997,199,26452,480.0,520.0,1352558.875,-1.0,1.001812,23978380.0,1.000859,1.001494,...,-0.000317,-0.000634,-0.000476,-0.000561,-0.000317,-0.000159,-0.000244,0.000158,0.000073,-0.000085
5290998,199,26453,480.0,530.0,1352558.875,-1.0,1.001812,23978380.0,1.000859,1.001494,...,-0.000317,-0.000634,-0.000476,-0.000566,-0.000317,-0.000159,-0.000249,0.000158,0.000068,-0.000090


## Group by stock_id

TODO: group by date_id too? or use embedding for date_id?

In [26]:
# df_grouped = df.groupby("date_id")
# num_classes = len(df_grouped)
# print(num_classes)
# print(df_grouped.size())

### Separate df_grouped into training and validation set by time-series k-fold index

In [27]:
# df_grouped_train_map = {}
# df_grouped_val_map = {}
# df_grouped_train_map_sizes_sum = 0
# df_grouped_val_map_sizes_sum = 0

# for item_id, gdf in df_grouped:
#     training_k_fold_idx_filter_mask = gdf.index.isin(k_fold_train_df_index)

#     training_df = gdf[training_k_fold_idx_filter_mask]
#     df_grouped_train_map[item_id] = training_df
#     df_grouped_train_map_sizes_sum += training_df.shape[0]

#     validation_df = gdf[~training_k_fold_idx_filter_mask]
#     df_grouped_val_map[item_id] = validation_df
#     df_grouped_val_map_sizes_sum += validation_df.shape[0]

#     assert training_df.shape[0] > 0 \
#         and validation_df.shape[0] > 0 \
#         and training_df.shape[0] + validation_df.shape[0] == gdf.shape[0], f"{item_id} invalid shape, training_df: {training_df.shape}, validation_df: {validation_df.shape}"

# print(df_grouped_train_map_sizes_sum, df_grouped_val_map_sizes_sum, k_fold_train_df_index.shape, k_fold_val_df_index.shape)
# print(df_grouped_train_map_sizes_sum + df_grouped_val_map_sizes_sum - k_fold_train_df_index.shape[0] - k_fold_val_df_index.shape[0])

## Stock-based feature engineering

In [28]:
# normalize_columns = set([
#     "imbalance_size",
#     "matched_size",
#     "bid_size",
#     "ask_size",
# ])
# normalize_columns = list(normalize_columns.intersection(set(feat_dynamic_real)))
# print(normalize_columns)

In [29]:
# stock_processors = [
#     StockNormalizeFeaturesPreprocessor(normalize_columns),
# ]
# stock_processor = CompositeGroupedDataPreprocessor(stock_processors)

In [30]:
# stock_processor.fit(df_grouped_train_map)

In [31]:
# df_grouped_train_map = stock_processor.apply(df_grouped_train_map)
# df_grouped_val_map = stock_processor.apply(df_grouped_val_map)

In [32]:
# display(df_grouped_train_map[0])
# display(df_grouped_val_map[0])

## Final features

In [33]:
# update feat_dynamic_real if needed after per-stock feature engineering
feat_dynamic_real = feat_dynamic_real

## Prepare Pytorch datasets

In [34]:
class OptiverDataset(torch.utils.data.Dataset):
    def __init__(self, df, feature_names, category_cols, target_col, time_idx_col, num_time_pts, time_idx_offset):
        super().__init__()
        self.df = df
        self.features = df[feature_names]
        self.categories = df[category_cols]
        self.targets = df[target_col]
        self.time_idx = df[time_idx_col]
        self.num_time_pts = num_time_pts
        self.time_idx_offset = time_idx_offset

    def __len__(self):
        return self.num_time_pts

    def __getitem__(self, idx):
        time_idx_mask = self.time_idx == self.time_idx_offset + idx
        features = self.features[time_idx_mask]
        categories = self.categories[time_idx_mask]
        targets = self.targets[time_idx_mask]
        return features.values, categories.values, targets.values

In [35]:
# stock_training_datasets = []
# for item_id, gdf in df_grouped_train_map.items():
#     stock_training_datasets.append(StockTrainingDataset(gdf, feat_dynamic_real, "target", item_id, seq_len))

# stock_validation_datasets = []
# for item_id, gdf in df_grouped_val_map.items():
#     stock_validation_datasets.append(StockTrainingDataset(gdf, feat_dynamic_real, "target", item_id, seq_len))

# print(len(stock_training_datasets), len(stock_validation_datasets))

In [36]:
# full_training_dataset = torch.utils.data.ConcatDataset(stock_training_datasets)
# full_validation_dataset = torch.utils.data.ConcatDataset(stock_validation_datasets)
# print(len(full_training_dataset), len(full_validation_dataset))

In [37]:
val_cutoff_time_id = math.floor(26455 * 0.9)
df_train = df[df["time_id"] < val_cutoff_time_id]
df_eval = df[df["time_id"] >= val_cutoff_time_id]
print(df_train.shape, df_eval.shape, val_cutoff_time_id)

(4761800, 31) (529200, 31) 23809


In [38]:
full_training_dataset = OptiverDataset(df_train, feat_dynamic_real, "stock_id", "target", "time_id", val_cutoff_time_id, 0)
full_validation_dataset = OptiverDataset(df_eval, feat_dynamic_real, "stock_id", "target", "time_id", 26455 - val_cutoff_time_id, val_cutoff_time_id)
print(len(full_training_dataset), len(full_validation_dataset))

23809 2646


In [39]:
training_sampler = torch.utils.data.RandomSampler(full_training_dataset)
validation_sampler = torch.utils.data.SequentialSampler(full_validation_dataset)

In [40]:
training_dataloader = torch.utils.data.DataLoader(
    full_training_dataset,
    batch_size=training_batch_size,
    sampler=training_sampler,
    # https://pytorch.org/docs/stable/data.html#single-and-multi-process-data-loading
    # num_workers=4,
    # https://pytorch.org/docs/stable/data.html#memory-pinning
    # pin_memory=True,
)
validation_dataloader = torch.utils.data.DataLoader(
    full_validation_dataset,
    batch_size=validation_batch_size,
    sampler=validation_sampler,
    # num_workers=4,
    # pin_memory=True,
)

In [41]:
training_sample_batch = next(iter(training_dataloader))
print("training_sample_batch", training_sample_batch[0].size(), training_sample_batch[1].size())
print("training_sample_batch", training_sample_batch[0].type(), training_sample_batch[1].type())
validation_sample_batch = next(iter(validation_dataloader))
print("validation_sample_batch", validation_sample_batch[0].size(), validation_sample_batch[1].size())
print("validation_sample_batch", validation_sample_batch[0].type(), validation_sample_batch[1].type())

training_sample_batch torch.Size([256, 200, 28]) torch.Size([256, 200])
training_sample_batch torch.FloatTensor torch.LongTensor
validation_sample_batch torch.Size([256, 200, 28]) torch.Size([256, 200])
validation_sample_batch torch.FloatTensor torch.LongTensor


## Create model

In [42]:
model = TransformerModel(
    num_input_features=num_input_features,
    num_classes=num_classes,
    embedding_dim=embedding_dim,
    d_model=d_model,
    nhead=nhead,
    d_hid=d_hid,
    nlayers=nlayers,
    dropout=dropout,
)
model.apply(init_weights)

TransformerModel(
  (embedding): Embedding(200, 4)
  (input_ff): Linear(in_features=32, out_features=32, bias=True)
  (input_ff_sigmoid): Sigmoid()
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
        )
        (linear1): Linear(in_features=32, out_features=32, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=32, out_features=32, bias=True)
        (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (final_linear): Linear(in_features=32, out_features=1, bias=True)
)

In [43]:
criterion = nn.L1Loss()
validation_criterion = nn.L1Loss()

## Tensorboard

Start with `tensorboard --logdir=lightning_logs/` cmd.

In [44]:
# %reload_ext tensorboard
# %tensorboard --logdir=lightning_logs/

## Module and trainer (lightning)

In [45]:
class TransformerModelModule(L.LightningModule):
    def __init__(
        self,
        model: nn.Module,
        criterion: nn.Module,
        validation_criterion: nn.Module,
        lr,
    ):
        super().__init__()
        self.save_hyperparameters()
        self.model = model
        self.criterion = criterion
        self.validation_criterion = validation_criterion
        self.lr = lr
        self.validation_step_outputs = []
        self.validation_step_actual_targets = []

    def training_step(self, batch, batch_idx):
        features, item_id, targets = batch[0], batch[1], batch[2]
        actual_targets = targets
        output = self.model(features, item_id)
        loss = self.criterion(output, actual_targets)
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        features, item_id, targets = batch[0], batch[1], batch[2]
        actual_targets = targets
        output = self.model(features, item_id)
        # TODO: is loss being averaged based on batch size
        loss = self.validation_criterion(output, actual_targets)
        # lightning will take weighted-average on loss per step based on batch size
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.validation_step_outputs.append(output)
        self.validation_step_actual_targets.append(actual_targets)

    def on_validation_epoch_end(self):
        # TODO: remove manual calculation of validation loss if we can confirm lightning will take weighted average
        # cat is used instead of stack, last step may have different batch size
        all_preds = torch.cat(self.validation_step_outputs)
        all_actual_targets = torch.cat(self.validation_step_actual_targets)
        manual_loss = self.validation_criterion(all_preds, all_actual_targets)
        self.log("val_loss_manual", manual_loss, on_step=False, on_epoch=True, prog_bar=True)
        self.validation_step_outputs.clear()  # free memory
        self.validation_step_actual_targets.clear()  # free memory

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.model.parameters(),
            lr=self.lr,
            betas=(0.9, 0.95),
            weight_decay=1e-1,
        )
        return [optimizer]

In [46]:
model = TransformerModelModule(
    model,
    criterion,
    validation_criterion,
    learning_rate,
)

/userhome/cs2/tsangsyf/anaconda3_2/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
/userhome/cs2/tsangsyf/anaconda3_2/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'criterion' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['criterion'])`.
/userhome/cs2/tsangsyf/anaconda3_2/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'validation_criterion' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['validation_criterion'])`.


In [47]:
checkpoint_callback = ModelCheckpoint(
    dirpath=checkpoint_dir,
    save_top_k=2,
    monitor="val_loss",
    mode="min",
    filename="transformer-{epoch:02d}-{val_loss:.4f}"
)

In [48]:
trainer = L.Trainer(
    max_epochs=num_epochs,
    limit_train_batches=limit_train_batches,
    # https://lightning.ai/docs/pytorch/stable/advanced/training_tricks.html#gradient-clipping
    gradient_clip_val=gradient_clip_val,
    callbacks=[
        # https://lightning.ai/docs/pytorch/stable/common/progress_bar.html#richprogressbar
        RichProgressBar(leave=True),
        checkpoint_callback,
    ],
    logger=tb_logger,
    # https://lightning.ai/docs/pytorch/stable/common/trainer.html#reproducibility
    deterministic=True,
)
print(trainer.callbacks)

/userhome/cs2/tsangsyf/anaconda3_2/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /userhome/cs2/tsangsyf/anaconda3_2/lib/python3.11/si ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..


[<lightning.pytorch.callbacks.progress.rich_progress.RichProgressBar object at 0x14826462cb90>, <lightning.pytorch.callbacks.rich_model_summary.RichModelSummary object at 0x14824f1dc590>, <lightning.pytorch.callbacks.model_checkpoint.ModelCheckpoint object at 0x148264668110>]


In [49]:
trainer.validate(
    model=model,
    dataloaders=validation_dataloader,
)

/userhome/cs2/tsangsyf/anaconda3_2/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /userhome/cs2/tsangsyf/anaconda3_2/lib/python3.11/si ...
2024-04-25 23:39:55.911865: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-25 23:40:01.139387: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

/userhome/cs2/tsangsyf/anaconda3_2/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │     6.066676616668701     │
│      val_loss_manual      │     6.066676616668701     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 6.066676616668701, 'val_loss_manual': 6.066676616668701}]

## Training

In [50]:
print(checkpoint_dir)
trainer.fit(
    model=model,
    train_dataloaders=training_dataloader,
    val_dataloaders=validation_dataloader,
)

/userhome/cs2/tsangsyf/anaconda3_2/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /userhome/cs2/tsangsyf/anaconda3_2/lib/python3.11/si ...
/userhome/cs2/tsangsyf/anaconda3_2/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:652: Checkpoint directory /userhome/cs2/tsangsyf/optiver2023/florence/transformer_encoder_by_time_checkpoints/20240425_transformer_dupletstriplets exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name                 ┃ Type             ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ model                │ TransformerModel │ 14.8 K │
│ 1 │ criterion            │ L1Loss           │      0 │
│ 2 │ validation_criterion │ L1Loss           │      0 │
└───┴──────────────────────┴──────────────────┴────────┘

Trainable params: 14.8 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 14.8 K                                                                                               
Total estimated model params size (MB): 0

Output()

/userhome/cs2/tsangsyf/anaconda3_2/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector
.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value
of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.

...

Output()

Output()

Output()

Output()

`Trainer.fit` stopped: `max_epochs=10` reached.


In [51]:
trainer.validate(
    model=model,
    dataloaders=validation_dataloader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │     5.958928108215332     │
│      val_loss_manual      │     5.958928108215332     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 5.958928108215332, 'val_loss_manual': 5.958928108215332}]

In [52]:
model

TransformerModelModule(
  (model): TransformerModel(
    (embedding): Embedding(200, 4)
    (input_ff): Linear(in_features=32, out_features=32, bias=True)
    (input_ff_sigmoid): Sigmoid()
    (transformer_encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-1): 2 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
          )
          (linear1): Linear(in_features=32, out_features=32, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=32, out_features=32, bias=True)
          (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (final_linear): Linear(in_features=32, out_features=1, bias=True)


## Load saved checkpoint

In [53]:
checkpoint_callback.best_model_path

'/userhome/cs2/tsangsyf/optiver2023/florence/transformer_encoder_by_time_checkpoints/20240425_transformer_dupletstriplets/transformer-epoch=06-val_loss=5.9584-v1.ckpt'

In [54]:
checkpoint_path = checkpoint_callback.best_model_path
print(checkpoint_path)

/userhome/cs2/tsangsyf/optiver2023/florence/transformer_encoder_by_time_checkpoints/20240425_transformer_dupletstriplets/transformer-epoch=06-val_loss=5.9584-v1.ckpt


In [55]:
saved_model = TransformerModelModule.load_from_checkpoint(checkpoint_path)
print(saved_model)

TransformerModelModule(
  (model): TransformerModel(
    (embedding): Embedding(200, 4)
    (input_ff): Linear(in_features=32, out_features=32, bias=True)
    (input_ff_sigmoid): Sigmoid()
    (transformer_encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-1): 2 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
          )
          (linear1): Linear(in_features=32, out_features=32, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=32, out_features=32, bias=True)
          (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (final_linear): Linear(in_features=32, out_features=1, bias=True)


/userhome/cs2/tsangsyf/anaconda3_2/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
/userhome/cs2/tsangsyf/anaconda3_2/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'criterion' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['criterion'])`.
/userhome/cs2/tsangsyf/anaconda3_2/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'validation_criterion' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['validation_criterion'])`.


In [56]:
torch_checkpoint = torch.load(checkpoint_path)
print(torch_checkpoint.keys())
print(torch_checkpoint["state_dict"].keys())
print(torch_checkpoint["hyper_parameters"])

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'hparams_name', 'hyper_parameters'])
odict_keys(['model.embedding.weight', 'model.input_ff.weight', 'model.input_ff.bias', 'model.transformer_encoder.layers.0.self_attn.in_proj_weight', 'model.transformer_encoder.layers.0.self_attn.in_proj_bias', 'model.transformer_encoder.layers.0.self_attn.out_proj.weight', 'model.transformer_encoder.layers.0.self_attn.out_proj.bias', 'model.transformer_encoder.layers.0.linear1.weight', 'model.transformer_encoder.layers.0.linear1.bias', 'model.transformer_encoder.layers.0.linear2.weight', 'model.transformer_encoder.layers.0.linear2.bias', 'model.transformer_encoder.layers.0.norm1.weight', 'model.transformer_encoder.layers.0.norm1.bias', 'model.transformer_encoder.layers.0.norm2.weight', 'model.transformer_encoder.layers.0.norm2.bias', 'model.transformer_encoder.layers.1.self_attn.in_proj_weight', 'model.transformer_

In [57]:
saved_model.eval()
trainer.validate(
    model=model,
    dataloaders=validation_dataloader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │     5.958928108215332     │
│      val_loss_manual      │     5.958928108215332     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 5.958928108215332, 'val_loss_manual': 5.958928108215332}]